In [30]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import Counter
from gensim.test.utils import datapath
import pickle
from sklearn.preprocessing import MinMaxScaler
from gensim.models import word2vec
import re 
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
import nltk
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [28]:
fulltrainingSet = pd.read_pickle('result.pickle')
trainingSet = fulltrainingSet
trainingSet = trainingSet.set_index("Id")

In [6]:
test_index = pd.read_csv("test.csv")['Id'].tolist()

In [7]:
test_set = trainingSet.iloc[test_index,:]

In [8]:
print(trainingSet.iloc[0,:])

ProductId                                             0005019281
UserId                                             ADZPIG9QOCDG5
Score                                                          4
Time                                                  1203984000
Summary                                good version of a classic
Text           This is a charming version of the classic Dick...
Helpfulness                                                    0
reviews        good version of a classicThis is a charming ve...
result                                                   4.40565
Name: 0, dtype: object


In [ ]:
#d = trainingSet[['UserId','ProductId']]
#d = d.apply(LabelEncoder().fit_transform, axis = 0)
#trainingSet['UserId'] = d['UserId']
#trainingSet['ProductId'] = d['ProductId']
#trainingSet = trainingSet[['ProductId','UserId','Score','Time','']]

In [9]:
UserId_group = trainingSet[['UserId','Score','result']]
def show_all_result(x):
    return [i for i in x]
UserId_group = UserId_group.groupby(['UserId']).agg({'Score':lambda x: show_all_result(x), 'result': lambda x: show_all_result(x)})

In [10]:
ProductId_group = trainingSet[['ProductId','Score','result']]
def show_all_result(x):
    return [i for i in x]
ProductId_group = ProductId_group.groupby(['ProductId']).agg({'Score':lambda x: show_all_result(x), 'result': lambda x: show_all_result(x)})

In [11]:
#test_set_user = test_set.merge(UserId_group, on='UserId', how='left')
#test_set_product = test_set.merge(ProductId_group, on='ProductId', how='left')
train_set_user = fulltrainingSet.merge(UserId_group, on='UserId', how='left')
train_set_product = fulltrainingSet.merge(ProductId_group, on='ProductId', how='left')

In [ ]:
print(train_set_user.shape)

In [12]:
train_set_user = train_set_user[['UserId','Score_y','result_y','result_x']]
sub_user = train_set_user[['Score_y','result_y']]
sub_user = sub_user.apply(lambda x: np.array([i for i in x]), axis = 1)

train_set_product = train_set_product[['UserId','Score_y','result_y','result_x']]
sub_product = train_set_product[['Score_y','result_y']]
sub_product = sub_product.apply(lambda x: np.array([i for i in x]), axis = 1)

In [ ]:
#test_set_product = test_set_product[['HelpfulnessNumerator','HelpfulnessDenominator','ProductId','Score_y','result_y','result_x']]
#sub_product = test_set_product[['Score_y','result_y']]
#sub_product = sub_product.apply(lambda x: np.array([i for i in x]), axis = 1)
#sub_help = test_set_product[['HelpfulnessNumerator','HelpfulnessDenominator']]

In [13]:
def get_difference(lst):
    lst = np.array(lst).transpose()
    lst = lst[~np.isnan(lst).any(axis = 1)]
    return np.mean(lst[:,0] - lst[:,1])
sub_user = sub_user.apply(lambda x: get_difference(x))
sub_product = sub_product.apply(lambda x: get_difference(x))

/Users/lw/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [14]:
sub_user[sub_user != sub_user] = 0
sub_product[sub_product != sub_product] = 0
#result = s1['result_x'] + sub_s1 + sub_x1

In [ ]:
print(sub_user)
print(sub_product)

In [15]:
fulltrainingSet['product'] = sub_product
fulltrainingSet['user'] = sub_user
print(fulltrainingSet)

              Id   ProductId          UserId  Score        Time  \
0              0  0005019281   ADZPIG9QOCDG5    4.0  1203984000   
1              1  0005019281  A35947ZP82G7JH    3.0  1388361600   
2              2  0005019281  A3UORV8A9D5L2E    3.0  1388361600   
3              3  0005019281  A1VKW06X1O2X7V    5.0  1202860800   
4              4  0005019281  A3R27T4HADWFFJ    4.0  1387670400   
...          ...         ...             ...    ...         ...   
1697528  1697528  B00LT1JHLW   AV657BUYHHXZ2    NaN  1406073600   
1697529  1697529  B00LT1JHLW  A17W587EH23J0Q    5.0  1405641600   
1697530  1697530  B00LT1JHLW  A3DE438TF1A958    5.0  1405728000   
1697531  1697531  B00LT1JHLW  A2RWCXDMANY0LW    5.0  1405987200   
1697532  1697532  B00LT1JHLW  A3ROPC55BE2OM9    5.0  1405728000   

                                                   Summary  \
0                                good version of a classic   
1                                   Good but not as moving   
2        

In [16]:
new_train= fulltrainingSet[['user','product','Helpfulness','result','Score','Time']].dropna()
new_train_y = new_train['Score']
new_train_x = new_train[['user','product','result','Helpfulness']]

In [17]:
new_train_x = new_train_x.to_numpy()
scaling = MinMaxScaler(feature_range=(0,1)).fit(new_train_x)
new_train_x = scaling.transform(new_train_x)
print(new_train_x)

[[0.53686615 0.59025367 0.85141329 0.        ]
 [0.4958468  0.59025367 0.810782   0.        ]
 [0.49207064 0.59025367 0.76949027 0.        ]
 ...
 [0.6501463  0.63443386 0.95825672 0.0375    ]
 [0.66387019 0.63443386 0.80454784 0.        ]
 [0.52948172 0.63443386 0.70655128 0.05978261]]


In [18]:
new_train_y = np.array(new_train_y)
print(new_train_y[new_train_y != new_train_y])

[]


In [19]:
X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y ,test_size=0.2)
clf = LogisticRegression(C = 0.01,solver ='saga', max_iter = 1000)
clf = clf.fit(new_train_x, new_train_y)
print(clf.score(X_test,y_test))
#clf = svm.SVC(kernel = "linear", gamma = 'auto')
#clf.fit(X_train, y_train)

0.6476546204567327


In [20]:
test = fulltrainingSet.iloc[test_index,:]
test = test[['user','product','result','Helpfulness']].to_numpy()
test = scaling.transform(test)
test_result = clf.predict_proba(test)

In [31]:
test_1 = np.matmul(clf.predict_proba(test),np.array([1,2,3,4,5]))
print(test_1)

[4.8605636  4.23100599 4.02969948 ... 3.55579802 4.88084766 4.30182434]


In [32]:
test_1[test_1 > 5] = 5
test_1[test_1 < 1] = 1
test_1 = np.around(test_1,0)


In [25]:
print(test_1)

[5. 4. 4. ... 4. 5. 4.]


In [26]:
x = pd.DataFrame({'Id':np.array(test_index),'Score':test_1})
x.to_csv("result_10.csv",index = False)
